In [1]:
import numpy as np

X = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("True generative process")

In [ ]:
rng = np.random.RandomState(2137)
training_indices = rng.choice(np.arange(y.size), size=20, replace=False)
X_train, y_train = X[training_indices], y[training_indices]
training_indices

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1 * RBF()
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(X_train, y_train)
gaussian_process.kernel_

In [ ]:
y_pred, std_prediction = gaussian_process.predict(X, return_std=True)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.scatter(X_train, y_train, label="Observations")
plt.plot(X, y_pred, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    y_pred - 1.96 * std_prediction,
    y_pred + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")

In [16]:
import seaborn as sns

In [ ]:
sns.lineplot(std_prediction)

In [110]:
from sklearn.model_selection import LeaveOneOut

training_indices = rng.choice(np.arange(y.size), size=5, replace=False)
X_train, y_train = X[training_indices], y[training_indices]
loo = LeaveOneOut()
loo.get_n_splits(X_train)

5

In [111]:
import warnings

from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [112]:
y_preds = []

for i, (train_index, test_index) in enumerate(loo.split(X_train)):

    kernel = 1 * RBF()
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    gaussian_process.fit(X_train[train_index], y_train[train_index])
    y_pred, std_prediction = gaussian_process.predict(
        X_train[test_index], return_std=True
    )
    true = y_train[test_index]

    print(f"For {X_train[test_index]} we predicted {y_pred} but True value is: {true}")
    y_preds.append(y_pred)

For [[8.41841842]] we predicted [7.75511031] but True value is: [7.11263771]
For [[8.01801802]] we predicted [6.67228363] but True value is: [7.91038554]
For [[4.4044044]] we predicted [2.88848759] but True value is: [-4.19716158]
For [[2.08208208]] we predicted [2.07639412] but True value is: [1.81581733]
For [[2.4024024]] we predicted [0.00019287] but True value is: [1.61847379]


In [113]:
y_preds = np.array(y_preds).flatten()

In [114]:
# y_preds, y_train

In [115]:
from sklearn.metrics import r2_score


r2_score(y_train, y_preds)

0.42891832577375866